## Preperation: load dependencies

In [2]:
//import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
//import $ivy.`com.typesafe.slick::slick:3.3.3`
//import $ivy.`com.typesafe.slick::slick-hikaricp:3.3.3`
//import $ivy.`com.github.tminglei::slick-pg:0.19.5`
//import $ivy.`com.github.tminglei::slick-pg_spray-json:0.19.5`
//import $ivy.`org.postgresql:postgresql:42.2.19`
//import $ivy.`ch.qos.logback:logback-classic:1.2.3`

// Load fat JAR
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu-assembly-0.1.jar"))

In [3]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

import $ivy.$                                       


import scala.language.postfixOps

import org.apache.commons.csv.CSVFormat

import smile._

import smile.util._

import smile.math._

import smile.math.MathEx._

import smile.math.kernel._

import smile.math.matrix._

import smile.math.matrix.Matrix._

import smile.math.rbf._

import smile.stat.distribution._

import smile.data._

import smile.data.formula._

import smile.data.measure._

import smile.data.`type`._

import smile.json._

import smile.interpolation._

import smile.validation._

import smile.association._

import smile.base.cart.SplitRule

import smile.base.mlp._

import smile.base.rbf.RBF

import smile.classification._

import smile.regression._

import smile.feature._

import smile.clustering._

import smile.vq._

import smile.manifold._

import smile.mds._

import smile.sequence._

import smile.projection._

import smile.nlp._

import smile.wavelet._
// Plotting

import smile.plot.swing._

import smile.plot.vega._

i

## Get data from DB

In [4]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.event.EventDB


val workflowListDB = new WorkflowListDB()
val eventDB = new EventDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val events =  Await.result(eventDB.getEvents, 2 seconds)

15:03:36.430 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding uuid -> java.util.UUID 
15:03:36.441 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding text -> java.lang.String 
15:03:36.450 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding bool -> Boolean 
15:03:36.584 [scala-interpreter-1] INFO  com.zaxxer.hikari.HikariDataSource - workflow.postgres.db - Starting...
15:03:36.625 [scala-interpreter-1] INFO  com.zaxxer.hikari.HikariDataSource - workflow.postgres.db - Start completed.


import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.event.EventDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@6772d83b
eventDB: EventDB = de.tmrdlt.database.event.EventDB@39e01a87
workflowLists: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-15T14:07:05.064
  ),
  WorkflowList(
    2L,
    "5fad0555e165558a64ad0479",
    "Tasks / ToDos",
    None,
    Some(1L),
    0L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-15T14:07:05.064
  ),
  WorkflowList(
    3L,
    "5fad0555e1655

# Trello
## Size of boards and columns over time

In [5]:
import de.tmrdlt.models._
import de.tmrdlt.database.workflowlist._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists
    .filter(_.listType == WorkflowListType.BOARD)
    .filter(_.dataSource == WorkflowListDataSource.Trello)
print(boards.length)

2

import de.tmrdlt.models._

import de.tmrdlt.database.workflowlist._

import java.time.LocalDate

import scala.collection.mutable.ListBuffer


boards: Seq[WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-15T14:07:05.064
  ),
  WorkflowList(
    41L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-15T14:07:05.064
  )
)

In [10]:
def plotDataForBoard(board: WorkflowList) = {
    val data = new ListBuffer[(Int, LocalDate)]
    data += ((0, board.createdAt.toLocalDate))
    
    events
        .filter(e => (e.eventType == "createCard" || e.eventType == "deleteCard") 
               && e.boardApiId == Some(board.apiId))
        .sortBy(_.date)
        .foreach { e => 
            val count = if (e.eventType == "createCard") data.last._1 + 1 else data.last._1 - 1
            data += ((count, e.date.toLocalDate))
        }
    
    val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
    val chart = VegaLite.view()
        .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
        .width(800)
        .height(200)
        .x(field = "date", `type` = "temporal")
        .y(field = "count", `type` = "quantitative")
        .data(smileData)
        .title(s"Size of the board '${board.title}' over time")
    show(chart)
    
    val columns = workflowLists
    .filter(_.listType == WorkflowListType.LIST)
    .filter(_.parentId == Some(board.id))
    .sortBy(_.position)
    val columnData = 
    columns.map { column =>
        val data = new ListBuffer[(Int, LocalDate, String, Int)]
        data += ((0, column.createdAt.toLocalDate, column.title, column.position.toInt))
        
        events
            .filter(e => e.parentApiId == Some(column.apiId) ||
                         e.oldParentApiId == Some(column.apiId) ||
                         e.newParentApiId == Some(column.apiId))
            .sortBy(_.date)
            .foreach { a => 
                val count = 
                    if (e.actionType == "createCard") {
                        data.last._1 + 1
                    } else if (a.actionType == "deleteCard") {
                        data.last._1 - 1  
                    } else {
                        if (column.apiId == a.oldParentApiId.get) {
                            data.last._1 - 1
                        } else {
                            data.last._1 + 1
                        }
                    } 
                data += ((count, a.date.toLocalDate, column.title, column.position.toInt))
            }
        data
    }
    
    columnData.foreach { data =>
        val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
        val chart = VegaLite.view()
            .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
            .width(800)
            .height(200)
            .x(field = "date", `type` = "temporal")
            .y(field = "count", `type` = "quantitative")
            .data(smileData)
            .title(s"Size of column '${data(0)._3}' over time")
        show(chart)
    }
    // Plot as Facet
    //val columnSmileData = columnData.flatten.toList.map(d => JsObject("card_count" -> JsInt(d._1), "date" -> JsDate(d._2), "column_name" -> JsString(d._3), "position" -> JsInt(d._4)))
    //val chart2 = VegaLite.facet(columnSmileData)
    //    .row(field = "column_name", `type` = "nominal")
    //    .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
    //    .width(800)
    //    .height(150)
    //    .x(field = "date", `type` = "temporal")
    //    .y(field = "card_count", `type` = "quantitative")
    //    .title(s"Size of the columns of board '${board.title}' over time")
    //show(chart2)
}

defined function plotDataForBoard

In [18]:
plotDataForBoard(boards(3))

<iframe id="e13f7d5c-848e-4a86-8650-cc337853e372" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2012-04-20","count":0},{"date":"2012-04-20","count":1},{"date":"2012-04-20","count":2},{"date":"2012-04-20","count":3},{"date":"2012-04-20","count":4},{"date":"2012-04-20","count":5},{"date":"2012-04-20","count":6},{"date":"2012-04-26","count":7},{"date":"2012-04-26","count":8},{"date":"2012-04-26","count":9},{"date":"2012-04-26","count":10},{"date":"2012-04-26","count":11},{"date":"2012-04-26","count":12},{"date":"2012-04-26","count":13},{"date":"2012-04-26","count":14},{"date":"2012-04-26","count":15},{"date":"2012-04-26","count":16},{"date":"2012-04-26","count":17},{"date":"2012-04-26","count":18},{"date":"2012-04-26","count":19},{"date":"2012-04-26","count":20},{"date":"2012-04-26","count":21},{"date":"2012-04-26","count":22},{"date":"2012-04-26","count":23},{"date":"2012-04-26","count":24},{"date":"2012-04-26","count":25},{"date":"2012-04-26","count":26},{"date":"2012-04-26","count":27},{"date":"2012-04-26","count":28},{"date":"2012-04-26","count":29},{"date":"2012-04-26","count":30},{"date":"2012-04-26","count":31},{"date":"2012-04-26","count":32},{"date":"2012-04-26","count":33},{"date":"2012-04-26","count":34},{"date":"2012-04-26","count":35},{"date":"2012-04-26","count":36},{"date":"2012-04-26","count":37},{"date":"2012-04-26","count":38},{"date":"2012-04-26","count":39},{"date":"2012-04-26","count":40},{"date":"2012-04-26","count":41},{"date":"2012-04-26","count":42},{"date":"2012-04-26","count":43},{"date":"2012-04-26","count":44},{"date":"2012-04-26","count":45},{"date":"2012-04-26","count":46},{"date":"2012-04-26","count":47},{"date":"2012-04-26","count":48},{"date":"2012-04-26","count":49},{"date":"2012-04-26","count":50},{"date":"2012-04-26","count":51},{"date":"2012-04-26","count":52},{"date":"2012-04-26","count":53},{"date":"2012-04-26","count":54},{"date":"2012-04-26","count":55},{"date":"2012-04-26","count":56},{"date":"2012-04-26","count":57},{"date":"2012-04-26","count":58},{"date":"2012-04-26","count":59},{"date":"2012-04-26","count":60},{"date":"2012-04-26","count":61},{"date":"2012-04-26","count":62},{"date":"2012-04-26","count":63},{"date":"2012-04-26","count":64},{"date":"2012-04-26","count":65},{"date":"2012-04-26","count":66},{"date":"2012-04-26","count":67},{"date":"2012-04-26","count":68},{"date":"2012-04-26","count":69},{"date":"2012-04-26","count":70},{"date":"2012-04-26","count":71},{"date":"2012-04-26","count":72},{"date":"2012-04-26","count":73},{"date":"2012-04-26","count":74},{"date":"2012-04-26","count":75},{"date":"2012-04-26","count":76},{"date":"2012-04-26","count":77},{"date":"2012-04-26","count":78},{"date":"2012-04-26","count":79},{"date":"2012-04-26","count":80},{"date":"2012-04-26","count":81},{"date":"2012-04-26","count":82},{"date":"2012-04-26","count":83},{"date":"2012-04-26","count":84},{"date":"2012-04-26","count":85},{"date":"2012-04-26","count":86},{"date":"2012-04-26","count":87},{"date":"2012-04-26","count":88},{"date":"2012-04-26","count":89},{"date":"2012-04-26","count":90},{"date":"2012-04-26","count":91},{"date":"2012-04-26","count":92},{"date":"2012-04-26","count":93},{"date":"2012-04-26","count":94},{"date":"2012-04-26","count":95},{"date":"2012-04-26","count":96},{"date":"2012-04-26","count":97},{"date":"2012-04-26","count":98},{"date":"2012-04-26","count":99},{"date":"2012-04-26","count":100},{"date":"2012-04-26","count":101},{"date":"2012-04-26","count":102},{"date":"2012-04-26","count":103},{"date":"2012-04-26","count":104},{"dat

<iframe id="0d781e71-3ecd-432b-afc9-757890ecaa0b" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2012-04-26","count":0},{"date":"2012-04-26","count":1},{"date":"2012-04-26","count":2},{"date":"2012-04-26","count":3},{"date":"2012-04-26","count":4},{"date":"2012-04-26","count":5},{"date":"2012-04-26","count":6},{"date":"2012-04-26","count":7},{"date":"2012-04-26","count":8},{"date":"2012-04-26","count":9},{"date":"2012-04-26","count":10},{"date":"2012-04-26","count":11},{"date":"2012-04-26","count":12},{"date":"2012-04-26","count":13},{"date":"2012-04-26","count":14},{"date":"2012-04-26","count":15},{"date":"2012-04-26","count":16},{"date":"2012-04-26","count":17},{"date":"2012-04-26","count":18},{"date":"2012-04-26","count":19},{"date":"2012-04-26","count":20},{"date":"2012-04-26","count":21},{"date":"2012-04-26","count":22},{"date":"2012-04-26","count":23},{"date":"2012-04-26","count":24},{"date":"2012-04-26","count":25},{"date":"2012-04-26","count":26},{"date":"2012-04-26","count":27},{"date":"2012-04-26","count":28},{"date":"2012-04-26","count":29},{"date":"2012-04-26","count":30},{"date":"2012-04-26","count":31},{"date":"2012-04-26","count":32},{"date":"2012-04-26","count":33},{"date":"2012-04-26","count":34},{"date":"2012-04-26","count":35},{"date":"2012-04-26","count":36},{"date":"2012-04-26","count":37},{"date":"2012-04-26","count":38},{"date":"2012-04-26","count":39},{"date":"2012-04-26","count":40},{"date":"2012-04-26","count":41},{"date":"2012-04-26","count":42},{"date":"2012-04-26","count":43},{"date":"2012-04-26","count":44},{"date":"2012-04-26","count":45},{"date":"2012-04-26","count":46},{"date":"2012-04-26","count":47},{"date":"2012-04-26","count":48},{"date":"2012-04-26","count":49},{"date":"2012-04-26","count":50},{"date":"2012-04-26","count":51},{"date":"2012-04-26","count":52},{"date":"2012-04-26","count":53},{"date":"2012-04-26","count":54},{"date":"2012-04-26","count":55},{"date":"2012-04-26","count":56},{"date":"2012-04-26","count":57},{"date":"2012-04-26","count":58},{"date":"2012-04-26","count":59},{"date":"2012-04-26","count":60},{"date":"2012-04-26","count":61},{"date":"2012-04-26","count":62},{"date":"2012-04-26","count":63},{"date":"2012-04-26","count":64},{"date":"2012-04-26","count":65},{"date":"2012-04-26","count":66},{"date":"2012-04-26","count":67},{"date":"2012-04-26","count":68},{"date":"2012-04-26","count":69},{"date":"2012-04-26","count":70},{"date":"2012-04-26","count":71},{"date":"2012-04-26","count":72},{"date":"2012-04-26","count":73},{"date":"2012-04-26","count":74},{"date":"2012-04-26","count":75},{"date":"2012-04-26","count":76},{"date":"2012-04-26","count":77},{"date":"2012-04-26","count":78},{"date":"2012-04-26","count":79},{"date":"2012-04-26","count":80},{"date":"2012-04-26","count":81},{"date":"2012-04-26","count":82},{"date":"2012-04-26","count":83},{"date":"2012-04-26","count":84},{"date":"2012-04-26","count":85},{"date":"2012-04-26","count":86},{"date":"2012-04-26","count":87},{"date":"2012-04-26","count":88},{"date":"2012-04-26","count":89},{"date":"2012-04-26","count":90},{"date":"2012-04-26","count":91},{"date":"2012-04-26","count":92},{"date":"2012-04-26","count":93},{"date":"2012-04-26","count":94},{"date":"2012-04-26","count":95},{"date":"2012-04-26","count":96},{"date":"2012-04-26","count":97},{"date":"2012-04-26","count":98},{"date":"2012-04-26","count":99},{"date":"2012-04-26","count":100},{"date":"2012-04-26","count":101},{"date":"2012-04-26","count":102},{"date":"2012-04-26","count":103},{"date":"2012-04-26","count":104},{"dat

<iframe id="f204057a-87fd-487f-9103-41884ccc978b" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2013-03-25","count":0},{"date":"2013-03-25","count":1},{"date":"2013-03-25","count":0},{"date":"2013-03-29","count":1},{"date":"2013-04-02","count":2},{"date":"2013-06-06","count":3},{"date":"2013-06-06","count":4},{"date":"2013-06-20","count":5},{"date":"2013-06-24","count":6},{"date":"2013-06-24","count":7},{"date":"2013-06-27","count":8},{"date":"2013-06-27","count":9},{"date":"2013-06-27","count":10},{"date":"2013-06-27","count":11},{"date":"2013-06-27","count":12},{"date":"2013-07-29","count":11},{"date":"2013-07-30","count":12},{"date":"2013-08-06","count":13},{"date":"2013-08-06","count":14},{"date":"2013-08-06","count":15},{"date":"2013-08-06","count":16},{"date":"2013-09-04","count":17},{"date":"2013-09-16","count":18},{"date":"2013-09-26","count":17},{"date":"2013-10-09","count":16},{"date":"2013-10-23","count":15},{"date":"2014-01-07","count":16},{"date":"2014-01-07","count":15},{"date":"2014-01-07","count":16},{"date":"2014-01-07","count":17},{"date":"2014-01-07","count":18},{"date":"2014-01-07","count":17},{"date":"2014-01-07","count":16},{"date":"2014-01-07","count":15},{"date":"2014-07-29","count":16},{"date":"2014-09-19","count":17},{"date":"2014-10-25","count":16},{"date":"2015-04-09","count":15},{"date":"2015-05-13","count":16},{"date":"2015-05-13","count":15},{"date":"2017-04-24","count":14}]},"width":800,"title":"Size of column 'Community Backlog' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="8053a504-1e2e-411e-a55b-0060018b6e0f" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2012-04-20","count":0},{"date":"2012-04-20","count":1},{"date":"2012-04-20","count":2},{"date":"2012-04-20","count":3},{"date":"2012-04-20","count":4},{"date":"2012-04-20","count":5},{"date":"2012-04-20","count":6},{"date":"2012-04-26","count":5},{"date":"2012-04-26","count":6},{"date":"2012-04-26","count":7},{"date":"2012-04-26","count":8},{"date":"2012-04-26","count":9},{"date":"2012-05-03","count":10},{"date":"2012-05-03","count":11},{"date":"2012-05-03","count":12},{"date":"2012-05-03","count":13},{"date":"2012-05-03","count":14},{"date":"2012-05-03","count":15},{"date":"2012-05-03","count":16},{"date":"2012-05-03","count":17},{"date":"2012-05-03","count":18},{"date":"2012-05-03","count":19},{"date":"2012-05-03","count":20},{"date":"2012-05-04","count":21},{"date":"2012-05-07","count":22},{"date":"2012-05-07","count":21},{"date":"2012-05-07","count":20},{"date":"2012-05-07","count":19},{"date":"2012-05-08","count":20},{"date":"2012-05-08","count":21},{"date":"2012-05-08","count":22},{"date":"2012-05-08","count":23},{"date":"2012-05-08","count":24},{"date":"2012-05-08","count":25},{"date":"2012-05-08","count":26},{"date":"2012-05-08","count":27},{"date":"2012-05-08","count":28},{"date":"2012-05-08","count":29},{"date":"2012-05-08","count":30},{"date":"2012-05-08","count":31},{"date":"2012-05-08","count":32},{"date":"2012-05-08","count":33},{"date":"2012-05-08","count":34},{"date":"2012-05-08","count":33},{"date":"2012-05-08","count":34},{"date":"2012-05-08","count":35},{"date":"2012-05-08","count":36},{"date":"2012-05-08","count":37},{"date":"2012-05-08","count":38},{"date":"2012-05-08","count":39},{"date":"2012-05-08","count":40},{"date":"2012-05-08","count":41},{"date":"2012-05-10","count":42},{"date":"2012-05-11","count":43},{"date":"2012-05-11","count":44},{"date":"2012-05-11","count":45},{"date":"2012-05-11","count":46},{"date":"2012-05-14","count":47},{"date":"2012-05-16","count":48},{"date":"2012-05-16","count":49},{"date":"2012-05-16","count":48},{"date":"2012-05-16","count":47},{"date":"2012-05-16","count":46},{"date":"2012-05-16","count":45},{"date":"2012-05-16","count":46},{"date":"2012-05-16","count":45},{"date":"2012-05-18","count":46},{"date":"2012-05-18","count":47},{"date":"2012-05-18","count":48},{"date":"2012-05-18","count":49},{"date":"2012-05-21","count":50},{"date":"2012-05-25","count":51},{"date":"2012-05-30","count":52},{"date":"2012-05-30","count":53},{"date":"2012-05-30","count":54},{"date":"2012-05-30","count":55},{"date":"2012-05-30","count":56},{"date":"2012-05-31","count":57},{"date":"2012-06-01","count":58},{"date":"2012-06-02","count":59},{"date":"2012-06-04","count":60},{"date":"2012-06-04","count":61},{"date":"2012-06-04","count":62},{"date":"2012-06-04","count":63},{"date":"2012-06-04","count":64},{"date":"2012-06-04","count":65},{"date":"2012-06-04","count":66},{"date":"2012-06-04","count":67},{"date":"2012-06-04","count":68},{"date":"2012-06-04","count":69},{"date":"2012-06-04","count":70},{"date":"2012-06-04","count":71},{"date":"2012-06-04","count":72},{"date":"2012-06-04","count":73},{"date":"2012-06-04","count":74},{"date":"2012-06-04","count":75},{"date":"2012-06-04","count":74},{"date":"2012-06-04","count":73},{"date":"2012-06-04","count":72},{"date":"2012-06-04","count":71},{"date":"2012-06-04","count":70},{"date":"2012-06-04","count":69},{"date":"2012-06-05","count":70},{"date":"2012-06-05","count":69},{"date":"2012-06-05","count":68},{"date":"201

<iframe id="3f38e006-98b6-40a5-bc53-9b0e70afdf8b" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2013-10-17","count":0},{"date":"2014-01-14","count":1},{"date":"2014-01-24","count":2},{"date":"2014-01-30","count":3},{"date":"2014-01-31","count":4},{"date":"2014-02-01","count":5},{"date":"2014-02-01","count":6},{"date":"2014-02-05","count":7},{"date":"2014-02-09","count":8},{"date":"2014-02-10","count":9},{"date":"2014-02-11","count":10},{"date":"2014-02-11","count":9},{"date":"2014-02-11","count":8},{"date":"2014-02-11","count":7},{"date":"2014-02-11","count":6},{"date":"2014-02-11","count":5},{"date":"2014-02-14","count":6},{"date":"2014-02-14","count":7},{"date":"2014-02-14","count":8},{"date":"2014-02-15","count":7},{"date":"2014-02-25","count":6},{"date":"2014-02-25","count":5},{"date":"2014-02-25","count":4},{"date":"2014-02-25","count":5},{"date":"2014-02-25","count":4},{"date":"2014-02-25","count":5},{"date":"2014-02-25","count":4},{"date":"2014-02-25","count":5},{"date":"2014-02-25","count":6},{"date":"2014-02-25","count":7},{"date":"2014-02-25","count":6},{"date":"2014-02-25","count":5},{"date":"2014-02-28","count":4},{"date":"2014-03-04","count":3},{"date":"2014-03-04","count":2},{"date":"2014-03-04","count":1},{"date":"2014-03-07","count":2},{"date":"2014-03-10","count":3},{"date":"2014-03-20","count":4},{"date":"2014-03-20","count":5},{"date":"2014-03-26","count":6},{"date":"2014-03-26","count":7},{"date":"2014-03-26","count":6},{"date":"2014-03-26","count":5},{"date":"2014-03-27","count":6},{"date":"2014-04-01","count":5},{"date":"2014-04-01","count":4},{"date":"2014-04-01","count":3},{"date":"2014-04-01","count":4},{"date":"2014-04-01","count":3},{"date":"2014-04-01","count":2},{"date":"2014-04-01","count":1},{"date":"2014-04-02","count":2},{"date":"2014-04-03","count":1},{"date":"2014-04-03","count":2},{"date":"2014-04-03","count":3},{"date":"2014-04-06","count":2},{"date":"2014-04-07","count":1},{"date":"2014-04-08","count":0},{"date":"2014-04-09","count":-1},{"date":"2014-04-11","count":-2},{"date":"2014-04-15","count":-3},{"date":"2014-04-15","count":-2},{"date":"2014-04-15","count":-1},{"date":"2014-04-16","count":-2},{"date":"2014-04-21","count":-1},{"date":"2014-04-22","count":0},{"date":"2014-04-22","count":1},{"date":"2014-04-22","count":2},{"date":"2014-04-22","count":3},{"date":"2014-04-23","count":4},{"date":"2014-04-23","count":5},{"date":"2014-04-24","count":6},{"date":"2014-04-24","count":7},{"date":"2014-04-25","count":6},{"date":"2014-04-25","count":7},{"date":"2014-04-28","count":8},{"date":"2014-05-01","count":7},{"date":"2014-05-02","count":8},{"date":"2014-05-05","count":9},{"date":"2014-05-09","count":10},{"date":"2014-05-12","count":11},{"date":"2014-05-13","count":12},{"date":"2014-05-13","count":11},{"date":"2014-05-13","count":10},{"date":"2014-05-13","count":9},{"date":"2014-05-13","count":8},{"date":"2014-05-14","count":9},{"date":"2014-05-15","count":10},{"date":"2014-05-20","count":11},{"date":"2014-05-20","count":10},{"date":"2014-05-20","count":9},{"date":"2014-05-20","count":8},{"date":"2014-05-20","count":7},{"date":"2014-05-20","count":6},{"date":"2014-05-20","count":7},{"date":"2014-05-20","count":6},{"date":"2014-05-20","count":5},{"date":"2014-05-20","count":4},{"date":"2014-05-20","count":3},{"date":"2014-05-20","count":2},{"date":"2014-05-20","count":3},{"date":"2014-05-20","count":4},{"date":"2014-05-22","count":5},{"date":"2014-05-22","count":6},{"date":"2014-05-23","count":7},{"date":"2014-05-30","count":6},{"date":"2014-05-30","cou

...

<iframe id="20b8abcf-1831-40cc-ae6d-585597963025" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2017-04-11","count":0},{"date":"2017-04-11","count":1},{"date":"2017-04-11","count":2},{"date":"2017-04-11","count":3},{"date":"2017-04-11","count":4},{"date":"2017-04-11","count":5},{"date":"2017-04-11","count":6},{"date":"2017-04-11","count":7},{"date":"2017-04-11","count":8},{"date":"2017-04-11","count":9},{"date":"2017-04-13","count":10},{"date":"2017-04-13","count":11},{"date":"2017-04-24","count":12},{"date":"2017-05-23","count":11},{"date":"2017-05-23","count":10},{"date":"2017-05-23","count":11},{"date":"2017-05-23","count":12},{"date":"2017-11-30","count":13}]},"width":800,"title":"Size of column 'Done - Brackets 1.10 Release' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="cd11ee81-70fe-459e-bab7-25bd919ca2fb" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2016-12-20","count":0},{"date":"2016-12-20","count":1},{"date":"2016-12-20","count":2},{"date":"2016-12-20","count":3},{"date":"2016-12-20","count":4},{"date":"2017-04-11","count":3},{"date":"2017-04-11","count":4},{"date":"2017-04-11","count":5},{"date":"2017-04-11","count":6}]},"width":800,"title":"Size of column 'Done - Brackets 1.9 Release' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="2beae6dc-3390-4338-b2fe-e1c727ed0df8" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2015-08-22","count":0},{"date":"2015-08-21","count":1},{"date":"2015-08-21","count":2},{"date":"2016-01-17","count":3}]},"width":800,"title":"Size of column 'Done - Release 1.4' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="05ec317e-1cc2-4192-83f7-905d4f056139" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2015-04-07","count":0},{"date":"2015-04-07","count":1},{"date":"2015-04-07","count":2},{"date":"2015-04-09","count":3},{"date":"2015-04-16","count":4},{"date":"2015-04-28","count":5},{"date":"2015-04-28","count":6}]},"width":800,"title":"Size of column 'Done - Release 1.3 - expected late April 2015' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="ba9faf73-fcc1-4fb5-bbf8-6913cf16e787" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2014-11-14","count":0},{"date":"2014-12-02","count":1},{"date":"2014-12-02","count":2},{"date":"2014-12-15","count":3},{"date":"2014-12-15","count":4},{"date":"2014-12-15","count":5},{"date":"2014-12-16","count":4},{"date":"2017-11-30","count":5}]},"width":800,"title":"Size of column 'Done - Release 1.1 - expected ~12/9' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">